In [1]:
import torch
from tensor import Tensor
import numpy as np 
from functools import partialmethod

# Testing tinygrad Code

In [17]:
x_init_ = np.random.randn(1, 3).astype(np.float32)
print(x_init_)
y_init_ = np.random.randn(3, 3).astype(np.float32)
print(y_init_)

Tx = Tensor(x_init_)
Tw = Tensor(y_init_)
out = Tx.mul(Tw)
print(out)
print(out.relu())

[[ 0.62641215 -1.2421117  -0.6285554 ]]
[[ 1.2792739  -0.4708473  -0.2693473 ]
 [-1.1053059  -0.8712512  -1.1002955 ]
 [ 0.4948434   1.621536    0.23808944]]
Tensor array([[ 0.8013527 ,  0.58484495,  0.1692997 ],
       [-0.69237703,  1.0821913 ,  0.69159675],
       [ 0.30997592, -2.014129  , -0.1496524 ]], dtype=float32) with grad None
Tensor array([[0.8013527 , 0.58484495, 0.1692997 ],
       [0.        , 1.0821913 , 0.69159675],
       [0.30997592, 0.        , 0.        ]], dtype=float32) with grad None


# Building Tensor 

In [3]:
class OSTensor:
    def __init__(self, data):
        self.data = data
        self.grad = None
    
    def __repr__(self) -> str:
        return f"OSTensor with data: {self.data}"

## Add Operators to the OSTensor 

In [4]:
# Crete the context class to save the data

class Context:
    def __init__(self, operatorClass, *osFunctionCls_and_data):
        self.operatorClass = operatorClass # arg
        self.osFunctionCls_and_data = osFunctionCls_and_data # tensors (tuple of - (<osFunction:class>, np.array[[1,2,4]]) )
        self.saved_tensors = []

    def __repr__(self) -> str:
        return f"Context: operatorClass - {self.operatorClass}, osFunctionCls_and_data - {self.osFunctionCls_and_data}"

    def save_for_backward(self, *data):
        self.saved_tensors.extend(data)


# This class implement the apply method which act as a factory method to register other oprtators
class osFunction:
    def apply(self, operatorClass, *data):
        contextObj = Context(operatorClass, self, *data) # the self in here is the osTensor object 
        val = operatorClass.forward(contextObj, self.data, *[t.data for t in data])
        t_ = OSTensor(val)
        return t_

## This method regiter operators into OSTensor class 

In [5]:
def register(name, operatorClass):
    setattr(OSTensor, name, partialmethod(operatorClass.apply, operatorClass))

## Tensor Multiplication

In [6]:
# Define the multiplication class and register it
class osMul(osFunction):
    @staticmethod
    def forward(contextObj, x, y):
        contextObj.save_for_backward(x, y)
        return x * y
    
register('mul', osMul)

### Test multiplications 

In [7]:
# Test the Multiplication
val1 = np.random.randn(1, 3).astype(np.float32)
val2 = np.random.randn(3, 3).astype(np.float32)

t1 = OSTensor(val1)
t2 = OSTensor(val2)
t3 = t1.mul(t2)
print(t2)

OSTensor with data: [[-2.026669   -0.9409733  -0.02389286]
 [-0.7585743  -0.2116443   1.3177452 ]
 [-1.0714103   0.511845   -0.7438869 ]]


## Dot operator (martix multiplications)

In [8]:
# Implement the metrix multipliation 

class osDot(osFunction):
    @staticmethod
    def forward(contextObj, input, weight):
        contextObj.save_for_backward(input, weight)
        return input.dot(weight)
    
register('dot', osDot)

### testing matrix multiplication

In [9]:
# test metrix multiplication

x_init_ = np.random.randn(1, 3).astype(np.float32)
y_init_ = np.random.randn(3, 3).astype(np.float32)

x = OSTensor(x_init_)
y = OSTensor(y_init_)

out = x.dot(y)
print(out)

OSTensor with data: [[ 1.0743885   0.48609254 -1.2785537 ]]


## ReLU operator

In [10]:
# Implement ReLU method 

class osReLU(osFunction):
    @staticmethod
    def forward(contextObj, input):
        contextObj.save_for_backward(input)
        return np.maximum(input, 0)
    
register('relu', osReLU)


### testing ReLU operator 

In [11]:
# test the ReLU method
print(out)
out2 = out.relu()
print(out2)

OSTensor with data: [[ 1.0743885   0.48609254 -1.2785537 ]]
OSTensor with data: [[1.0743885  0.48609254 0.        ]]


## Sum operator 

In [12]:
# Sum Tensors 
class osSum(osFunction):
    @staticmethod
    def forward(contextObj, input):
        contextObj.save_for_backward(input)
        return np.array(input.sum())
    
register('sum', osSum)

### testing Sum operator 

In [13]:
# test the sum 
out2.sum()

OSTensor with data: 1.560481071472168

## LogSoftMax operator 

In [14]:
# implement logsoftmax
class osLogSoftMax(osFunction):
    @staticmethod
    def forward(contextObj, input):
        contextObj.save_for_backward(input)
        def logsumexp(x):
            c = x.max(axis=1)
            return c + np.log(np.exp(x-c.reshape((-1, 1))).sum(axis=1))
        output = input - logsumexp(input).reshape((-1, 1))
        contextObj.save_for_backward(output)
        return output
    
register('logsoftmax', osLogSoftMax)

### testing LogSoftMax

In [15]:
# test the logsoftmax
out3 = out2.logsoftmax()
print(out3)

OSTensor with data: [[-0.6401572 -1.2284532 -1.7145457]]


# Testing the code and compare with tinygrad results

In [16]:
x_init = np.random.randn(1,3).astype(np.float32)
W_init = np.random.randn(3,3).astype(np.float32)
m_init = np.random.randn(1,3).astype(np.float32)

x = Tensor(x_init)
W = Tensor(W_init)
m = Tensor(m_init)

osx = OSTensor(x_init)
osw = OSTensor(W_init)
osm = OSTensor(m_init)

print(x.data)
print(osx.data)

assert np.array_equal(x.data, osx.data)
print('*'*10)
out = x.dot(W)
os_out = osx.dot(osw)

print(out.data)
print(os_out.data)

assert np.array_equal(out.data, os_out.data)

print('*'*10)
outr = out.relu()
os_outr = os_out.relu()

assert np.array_equal(outr.data, os_outr.data)

print('*'*10)
outl = outr.logsoftmax()
os_outl = os_outr.logsoftmax()

print(outl)
print(os_outl)
assert np.array_equal(outl.data, os_outl.data)

print('*'*10)
outm = outl.mul(m)
os_outm = os_outl.mul(osm)

print(outm)
print(os_outm)

assert np.array_equal(outm.data, os_outm.data)

print('*'*10)
outx = outm.sum()
os_outx = os_outm.sum()

print(outx)
print(os_outx)

assert np.array_equal(outx.data, os_outx.data)

# outx.backward()

# return outx.data, x.grad, W.grad



[[-1.9164127  0.709103  -1.1444359]]
[[-1.9164127  0.709103  -1.1444359]]
**********
[[3.4285579  0.42149386 3.7809215 ]]
[[3.4285579  0.42149386 3.7809215 ]]
**********
**********
Tensor array([[-0.9049716, -3.9120357, -0.552608 ]], dtype=float32) with grad None
OSTensor with data: [[-0.9049716 -3.9120357 -0.552608 ]]
**********
Tensor array([[ 5.380907e-03, -6.105719e+00,  5.690182e-01]], dtype=float32) with grad None
OSTensor with data: [[ 5.380907e-03 -6.105719e+00  5.690182e-01]]
**********
Tensor array(-5.5313196, dtype=float32) with grad None
OSTensor with data: -5.531319618225098


![Tensor1](img/tensor1.png)

![tensor2](img/tensor2.png)